In [2]:
import os
from typing import Union

from pydantic import BaseModel, Field

def get_llm():
    """
    Get the LlamaIndex Anthropic LLM.
    
    Set ANTHROPIC_API_KEY environment variable.
    
    Available models:
    - claude-sonnet-4-20250514 (recommended for most tasks)
    - claude-opus-4-20250514 (most capable)
    - claude-3-5-sonnet-20241022
    - claude-3-opus-20240229
    """
    from llama_index.llms.anthropic import Anthropic
    
    api_key = os.environ.get("ANTHROPIC_API_KEY")
    
    if not api_key:
        raise ValueError(
            "ANTHROPIC_API_KEY environment variable is required. "
            "Set it with: export ANTHROPIC_API_KEY=your-key-here"
        )
    
    model = os.environ.get("ANTHROPIC_MODEL", "claude-sonnet-4-20250514")
    
    return Anthropic(
        model=model,
        api_key=api_key,
        temperature=0.0,  # Deterministic for form filling
    )

In [11]:
def get_openai_llm():
    from llama_index.llms.openai import OpenAI
    return OpenAI(
        model="gpt-4o-mini",
        api_key=os.environ.get("OPENAI_API_KEY"),
    )


In [3]:
from pydantic import BaseModel, Field

class Texts(BaseModel):
    """Collection of texts."""
    texts: list[str] = Field(
        default_factory=list,
        description="List of text in natural language"
    )


In [ ]:
from llama_index.core.prompts import ChatPromptTemplate
from llama_index.core.llms import ChatMessage

user_topic = "The future of AI"
prompt = ChatPromptTemplate(
    message_templates=[
        ChatMessage.from_str(
            """You write an essay about the following topic:
            {user_topic}

            Write a list of texts.""",
            role="user"
        )
    ]
)

# Get LLM with structured output
# sllm = llm.as_structured_llm(output_cls=Texts)

In [ ]:
# ANTHROPIC: doesn't work
llm = get_llm()
# OPENAI: works
llm = get_openai_llm()

In [14]:
# Call with structured output
# response = sllm.complete(prompt)
llm.structured_predict(
    Texts, prompt, user_topic=user_topic
)

Texts(texts=['The future of AI: Opportunities and Challenges', 'How AI is Transforming Industries: A Look Ahead', "The Role of AI in Shaping Tomorrow's Workforce", 'Ethical Considerations in the Future of AI Development', 'AI and the Environment: Sustainable Solutions for a Greener Future', 'The Impact of AI on Healthcare: Innovations and Predictions', 'AI in Education: Revolutionizing Learning Experiences', 'The Future of AI in Transportation: Autonomous Vehicles and Beyond', 'AI and Creativity: Can Machines Become Artists?', 'The Global AI Race: Who Will Lead the Future?', 'AI and Privacy: Balancing Innovation with Personal Data Protection', 'The Future of AI in Finance: Automation and Risk Management', 'Human-AI Collaboration: Redefining Work in the 21st Century', 'The Role of AI in Addressing Global Challenges: Climate Change, Poverty, and More', 'The Future of AI Governance: Regulation and Policy Implications'])

In [ ]:
response

CompletionResponse(text='{"texts":[]}', additional_kwargs={}, raw=Texts(texts=[]), logprobs=None, delta=None)

In [7]:
response_base = llm.complete(prompt)
response_base

AnthropicCompletionResponse(text="# Essay: The Future of AI\n\n## Introduction\n\nArtificial Intelligence stands at the threshold of unprecedented transformation, poised to reshape every aspect of human civilization. As we advance deeper into the 21st century, AI's evolution from a theoretical concept to a practical reality has accelerated exponentially. The future of AI promises both extraordinary opportunities and significant challenges that will fundamentally alter how we work, live, and understand ourselves as a species.\n\n## Current State and Trajectory\n\nToday's AI landscape is dominated by machine learning algorithms, neural networks, and large language models that have already begun revolutionizing industries from healthcare to transportation. The rapid development of generative AI, autonomous systems, and sophisticated pattern recognition capabilities suggests we are merely witnessing the early stages of a technological revolution that will dwarf previous industrial transfor

In [ ]:
response_test = llm.structured_prompt(prompt, output_cls=Texts)